In [6]:
from functools import cmp_to_key
import numpy as np

def construct_sa(S):
    n = len(S)
    
    sa = np.zeros(n+1,dtype=int)
    sa[-1] = n
    rank = np.zeros(n+1,dtype=int)
    rank[-1] = -1
    #初始长度为1，rank 直接取字符串的编码
    for i in range(0,n):
        rank[i] = ord(S[i])
        sa[i] = i
    
    def cmpBaseRank(n,rank,k,i,j):
        if rank[i] != rank[j]:
            return 1 if rank[i] > rank[j] else -1
        else:
            ri = rank[i+k] if (i+k<=n) else -1
            rj = rank[j+k] if (j+k<=n) else -1
            return 1 if ri > rj else -1
        
    k = 1
    tmp = np.zeros(n+1,dtype=int)
    while k <= n:
        sa = sorted(sa, key=cmp_to_key(lambda i,j:cmpBaseRank(n, rank, k,i,j)))
        #由长度 k 的 rank 计算长度 2*k 的 rank
        tmp[sa[0]] = 0
        for i in range(1,n+1):
            tmp[sa[i]] = tmp[sa[i-1]] + (1 if (cmpBaseRank(n,rank,k,sa[i-1],sa[i]) == -1) else 0)
        rank = np.tile(tmp,1)
        k = k * 2

    return sa

print(construct_sa("abcabc"))

[6, 3, 0, 4, 1, 5, 2]


In [12]:
def construct_lcp(S, sa):
    n = len(S)
    rank = np.zeros(n+1, dtype=int)
    for i in range(0,n+1):
        rank[sa[i]] = i
    
    h = 0
    lcp = np.zeros(n,dtype=int)
    for i in range(0,n):
        j = sa[rank[i]-1]
        
        if h>0:
            h -= 1
        while j + h < n and i + h < n:
            if S[j+h] != S[i+h]:
                break
            h += 1
        lcp[rank[i] - 1] = h
    return lcp

In [14]:
S = "abracadabra"
sa = construct_sa(S)
lcp = construct_lcp(S, sa)
print(lcp)

[0 1 4 1 1 0 3 0 0 0 2]


In [42]:
def getCommanString(S,T):
    ST = S + '\0' + T
    
    sa = construct_sa(ST)
    lcp = construct_lcp(ST, sa)
    
    ans = 0
    pos = len(ST)
    n1 = len(S)
    for i in range(0,pos):
        if (sa[i] < n1) != (sa[i+1] < n1):
            if lcp[i] > ans:
                ans = lcp[i]
                pos = sa[i]
    #print(pos,ans)
    return ST[pos:pos+ans]

In [50]:
def test():
    S = "abcdAB"
    T = "abcdDD"
    print(getCommanString(S,T), getCommanString(S,T) == "abcd")
    
    S = "KLabcBDAD"
    T = "abcdBDAD"
    print(getCommanString(S,T), getCommanString(S,T) == "BDAD")
    
    S = "BDeios"
    T = "abcdBDAD"
    print(getCommanString(S,T), getCommanString(S,T) == "BD")
    
    S = ""
    T = ""
    print(getCommanString(S,T), getCommanString(S,T) == "")
    
    S = "ABDD"
    T = ""
    print(getCommanString(S,T), getCommanString(S,T) == "")
    
    S = ""
    T = "AB"
    print(getCommanString(S,T), getCommanString(S,T) == "")
    
    S = "AB"
    T = "AB"
    print(getCommanString(S,T), getCommanString(S,T) == "AB")
    
test()

abcd True
BDAD True
BD True
 True
 True
 True
AB True
